In [1]:
import numpy as np
import re

In [2]:
def loadDataSet():
    """
        创建数据和标签,这是一个用来区分是否有侮辱性文字的例子
        1表示言论中有侮辱性字样，0表示言论中没有侮辱性字样
    """
    postingList = [['my','dog','has','flea','problems','help','please'],
                   ['maybe','not','take','him','to','dog','park','stupid'],
                   ['my','dalmation','is','so','cute','I','love','him'],
                   ['stop','posting','stupid','worthless','garbage'],
                   ['mr','licks','ate','my','steak','how','to','stop','him'],
                   ['quit','buying','worthless','dog','food','stupid']
                  ]
    classVec = [0,1,0,1,0,1]
    return postingList,classVec

In [3]:
def createVocabList(dataSet):
    """
        将所有上面数据中出现的单词整合到一个set中，去掉重复的单词
        其中|表示的是两个Set取交集
    """
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet|set(document)
    return list(vocabSet)

In [7]:
def setOfWords2Vec(vocabList,inputSet):
    """
        将InputSet中的单词转化为向量
        returnVec是和vocabList一样长的list，如果inputSet中的单词存在voabList中，
        则标记为1存入returnVec中
    """
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)]=1
        else:
            print("the word:%s is not in my Vocabulary!"%word)
    return returnVec

In [8]:
listIPosts,listClasses = loadDataSet()
myVocabList = createVocabList(listIPosts)

In [9]:
myVocabList

['take',
 'love',
 'posting',
 'stop',
 'how',
 'quit',
 'stupid',
 'dog',
 'dalmation',
 'problems',
 'has',
 'not',
 'park',
 'so',
 'cute',
 'mr',
 'my',
 'please',
 'him',
 'I',
 'flea',
 'help',
 'maybe',
 'ate',
 'buying',
 'steak',
 'is',
 'garbage',
 'licks',
 'food',
 'to',
 'worthless']

In [10]:
setOfWords2Vec(myVocabList,listIPosts[0])

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

In [15]:
def trainNB0(trainMatrix,trainCategory):
    numTrainDocs = len(trainMatrix)
    numWords = len(trainMatrix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = np.ones(numWords)
    p1Num = np.ones(numWords)
    p0Denom = 2.0
    p1Denom = 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num/p1Denom)
    p0Vect = np.log(p0Num/p0Denom)
    return p0Vect,p1Vect,pAbusive

In [16]:
trainMat = []
for postinDoc in listIPosts:
    trainMat.append(setOfWords2Vec(myVocabList,postinDoc))

In [13]:
sum(trainMat[0])

7

In [17]:
p0V,p1V,pAb = trainNB0(trainMat,listClasses)

In [18]:
p0V

array([-3.25809654, -2.56494936, -3.25809654, -2.56494936, -2.56494936,
       -3.25809654, -3.25809654, -2.56494936, -2.56494936, -2.56494936,
       -2.56494936, -3.25809654, -3.25809654, -2.56494936, -2.56494936,
       -2.56494936, -1.87180218, -2.56494936, -2.15948425, -2.56494936,
       -2.56494936, -2.56494936, -3.25809654, -2.56494936, -3.25809654,
       -2.56494936, -2.56494936, -3.25809654, -2.56494936, -3.25809654,
       -2.56494936, -3.25809654])

In [19]:
p1V

array([-2.35137526, -3.04452244, -2.35137526, -2.35137526, -3.04452244,
       -2.35137526, -1.65822808, -1.94591015, -3.04452244, -3.04452244,
       -3.04452244, -2.35137526, -2.35137526, -3.04452244, -3.04452244,
       -3.04452244, -3.04452244, -3.04452244, -2.35137526, -3.04452244,
       -3.04452244, -3.04452244, -2.35137526, -3.04452244, -2.35137526,
       -3.04452244, -3.04452244, -2.35137526, -3.04452244, -2.35137526,
       -2.35137526, -1.94591015])

In [20]:
def classifyNB(vec2Classify,p0Vec,p1Vec,pClass1):
    p1 = sum(vec2Classify*p1Vec) + np.log(pClass1)
    p0 = sum(vec2Classify*p0Vec) + np.log(1.0 - pClass1)
    if p1>p0:
        return 1
    else:
        return 0

In [21]:
def testingNB():
    lisOPosts,listClasses = loadDataSet()
    myVocabList = createVocabList(lisOPosts)
    trainMat = []
    for postinDoc in lisOPosts:
        trainMat.append(setOfWords2Vec(myVocabList,postinDoc))
    p0v,p1v,pAb =trainNB0(np.array(trainMat),np.array(listClasses))
    testEntry = ['love','my','dalmation']
    thisDoc = np.array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'clasified as:',classifyNB(thisDoc,p0v,p1v,pAb))
    testEntry = ['stupid','garbage']
    thisDoc = np.array(setOfWords2Vec(myVocabList,testEntry))
    print(testEntry,'clasified as:',classifyNB(thisDoc,p0v,p1v,pAb))

In [22]:
testingNB()

['love', 'my', 'dalmation'] clasified as: 0
['stupid', 'garbage'] clasified as: 1


In [23]:
mySent = 'This book is the best book on Python or M.L. I have laid eyes upon.'

In [24]:
def textParse(bigString):
    regEx = re.compile('\\W*')
    listOfTokens = regEx.split(mySent)
    return [tok.lower() for tok in listOfTokens if len(tok)>2]

In [40]:
def spamTest():
    docList = []
    classList = []
    fullText = []
    for i in range(1,26):
        wordList = textParse(open('email/spam/%d.txt' %i).read())
        docList.append(wordList)
        fullText.append(wordList)
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt' %i).read())
        docList.append(wordList)
        fullText.append(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)
    trainSet = list(range(50))
    testSet = []
    for i in range(10):
        randIndex = int(np.random.uniform(0,len(trainSet)))
        testSet.append(trainSet[randIndex])
        del(trainSet[randIndex])
    trainMat=[]
    trainClasses = []
    for docIndex in trainSet:
        trainMat.append(setOfWords2Vec(vocabList,docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(np.array(trainMat),np.array(trainClasses))
    errcount = 0
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList,docList[docIndex])
        if classifyNB(np.array(wordVector),p0V,p1V,pSpam)!=classList[docIndex]:
            errcount+=1
    print("the error rate is:",float(errcount)/len(testSet))

In [41]:
spamTest()

the error rate is: 0.6


d:\python\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: split() requires a non-empty pattern match.
  This is separate from the ipykernel package so we can avoid doing imports until
